과적합(overfitting)이란,,,

학습이 이미 학습된 특정한 데이터에만 잘 예측하고, 다른 테스트용/실제 데이터에는 제대로 작동하지 않는 현상을 의미한다.

초음파를 통해 광물과 돌을 구분하는 sonar예제를 통해서 과적합 문제를 살펴보자.

아래 코드를 실행하면 매우 오래걸리지만 결국 언젠간 분석해낸다.
아래 데이터를 보면 20개의 샘플, 60개의 속성, 1개의 클래스로 구성되어있다.
참고로 마지막 클래스는 자료형이 class이기에 0과 1로 표현해줘야한다.

In [2]:
# 데이터 입력
from google.colab import files
uploaded = files.upload()
my_data = 'sonar.csv'

#본문에 맞는 텐서플로 버전을 선택합니다.
# !pip install -q tensorflow-gpu==1.15.0
# %tensorflow_version 1.x

import tensorflow as tf
import pandas as pd
import numpy

from keras.models import Sequential
from keras.layers.core import Dense
from sklearn.preprocessing import LabelEncoder

# seed 값 설정
numpy.random.seed(3)
tf.compat.v1.set_random_seed(3)

#데이터 적용
df = pd.read_csv(my_data, header=None)

# 데이터 개괄 보기
print(df.info())

# 데이터의 일부분 미리 보기
print(df.head())

Saving sonar.csv to sonar (1).csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208 entries, 0 to 207
Data columns (total 61 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       208 non-null    float64
 1   1       208 non-null    float64
 2   2       208 non-null    float64
 3   3       208 non-null    float64
 4   4       208 non-null    float64
 5   5       208 non-null    float64
 6   6       208 non-null    float64
 7   7       208 non-null    float64
 8   8       208 non-null    float64
 9   9       208 non-null    float64
 10  10      208 non-null    float64
 11  11      208 non-null    float64
 12  12      208 non-null    float64
 13  13      208 non-null    float64
 14  14      208 non-null    float64
 15  15      208 non-null    float64
 16  16      208 non-null    float64
 17  17      208 non-null    float64
 18  18      208 non-null    float64
 19  19      208 non-null    float64
 20  20      208 non-null    float64
 21  21   

In [ ]:
dataset = df.values
X = dataset[:,0:60]
Y_obj = dataset[:,60]

# 문자열 One-Hot Incoding
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

# 모델 설정
model = Sequential()
model.add(Dense(24,  input_dim=60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# 모델 컴파일
model.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['accuracy'])

# 모델 실행
model.fit(X, Y, epochs=200, batch_size=5)

# 결과 출력
print("\n Accuracy: %.4f" % (model.evaluate(X, Y)[1]))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Epoch 1/200
208/208 [==============================] - 0s 2ms/step - loss: 0.2469 - accuracy: 0.5625
Epoch 2/200
208/208 [==============================] - 0s 305us/step - loss: 0.2408 - accuracy: 0.5433
Epoch 3/200
208/208 [==============================] - 0s 312us/step - loss: 0.2269 - accuracy: 0.6154
Epoch 4/200
208/208 [==============================] - 0s 318us/step - loss: 0.2186 - accuracy: 0.6346
Epoch 5/200
208/208 [==============================] - 0s 293us/step - loss: 0.2100 - accuracy: 0.6875
Epoch 6/200
208/208 [==============================] - 0s 298us/step - loss: 0.2023 - accuracy: 0.7115
Epoch 7/200
208/208 [==============================] - 0s 249us/step - loss: 0.1941 - accuracy: 0.7404
Epoch 8/200
208/208 [==============================] - 0s 302us/step - loss: 0.1853 - accuracy: 0.7740
Epoch 9/200
208/208 [==============================] - 0s 291us/step - loss: 0.1770 - accuracy: 

100퍼센트의 정확도가 나온다.
과적합의 냄새가 풀풀 나지 않는가?

# 과적합은 왜 일어날까?

은닉층의 개수가 너무 많은 경우
변수가 복잡한 경우
테스트셋과 학습셋이 중복되는 경우
...
등 과적합이 일어나는 이유는 많다.

중요한 점은
**식이 복잡해지고 학습량이 늘어날 수록 학습 데이터셋에 대한 예측은 강화되지만, 그 외의 데이터셋에 대해서는 예측률이 떨어진다는 것이다.**

# 과적합을 방지하는 방법
학습 데이터와 테스트 데이터를 구분하고, 학습과 동시에 테스트를 병행하며 학습을 진행하는 방법이 있습니다.
그리고 학습을 진행해도 테스트 결과가 더 이상 좋아지지 않는 지점에서 학습을 멈추는 것입니다.